Importing packages

In [9]:
import scipy.integrate as scint
import numpy as np
import matplotlib.pyplot as plt

from typing import Optional, Iterable
flint = float | int
interval_t = tuple[flint, flint] | Iterable[flint]

Simulation function

In [10]:
def execute_simulation(f,
                       u0: Iterable[float | int],
                       coeffs: Iterable[float | int],
                       time_bound: float | int = 50,
                       granularity: Optional[int | str] = "auto"):
    # handle default values and make sure this function is effectively pass-by-value.
    u0_val = [item for item in u0].copy()
    coeffs_val = [item for item in coeffs].copy()
    time_bound_val = time_bound
    granularity_val = None if granularity == "auto" else granularity

    auto_evals = granularity_val is None
    if isinstance(granularity_val, str) and not auto_evals:
        raise ValueError(f"`granularity` must be either \"auto\" or a non-string, but was `{granularity}`.")

    t_span = (0, time_bound_val)
    y0 = u0_val
    args = (coeffs_val,)
    method = 'Radau'
    t_eval = None if auto_evals else np.linspace(0, time_bound_val, granularity_val + 1)

    # try to solve the ODE problem
    sol = None
    try:
        sol = scint.solve_ivp(fun=f, t_span=t_span, y0=y0, method=method, t_eval=t_eval, args=args)
    except Exception:
        print("fail")
        return (False, u0_val, coeffs_val, time_bound_val, granularity_val, sol)
    print("success")

    # return the results
    return (True, u0_val, coeffs_val, time_bound_val, granularity_val, sol)

Visualization function

In [11]:

def visualize_simulation(u0, coeffs, time_bound, granularity, sol):
    fig, axs = plt.subplots(3, 2, layout='constrained')
    try:
        t = sol.t
        p, q, x, y, r, s = sol.y
        p0, q0, x0, y0, r0, s0 = u0
        k1, k2, k3, k4 = coeffs
        the_label = f"u0=({p0}, {q0}, {x0}, {y0}, {r0}, {s0});\n{k1=}; {k2=}; {k3=}; {k4=}"
        axs[0, 0].plot(t, p, label=the_label)
        axs[0, 1].plot(t, q, label=the_label)
        axs[1, 0].plot(t, x, label=the_label)
        axs[1, 1].plot(t, y, label=the_label)
        axs[2, 0].plot(t, r, label=the_label)
        axs[2, 1].plot(t, s, label=the_label)
        axs[1, 0].plot(t, y, label=the_label, alpha=0.2)  # for comparison
        axs[1, 1].plot(t, x, label=the_label, alpha=0.2)  # for comparison
        # TODO: make the color of these lines appear in the legend too
        # TODO: make these lines appear behind the other ones, but with these colors
        # TODO: make it so the labels are aligned with one another
        axs[0, 0].set(ylabel="p")
        axs[0, 0].xaxis.set_major_formatter('')
        axs[0, 0].tick_params(axis='x', length=0)
        axs[0, 0].grid(True, linestyle='dashed')
        axs[0, 1].set(ylabel="q")
        axs[0, 1].xaxis.set_major_formatter('')
        axs[0, 1].tick_params(axis='x', length=0)
        axs[0, 1].grid(True, linestyle='dashed')
        axs[1, 0].set(ylabel="x")
        axs[1, 0].xaxis.set_major_formatter('')
        axs[1, 0].tick_params(axis='x', length=0)
        axs[1, 0].grid(True, linestyle='dashed')
        axs[1, 1].set(ylabel="y")
        axs[1, 1].xaxis.set_major_formatter('')
        axs[1, 1].tick_params(axis='x', length=0)
        axs[1, 1].grid(True, linestyle='dashed')
        axs[2, 0].set(ylabel="r", xlabel='t')
        axs[2, 0].grid(True, linestyle='dashed')
        axs[2, 1].set(ylabel="s", xlabel='t')
        axs[2, 1].grid(True, linestyle='dashed')
        p_min, p_max = axs[0, 0].get_ylim()
        q_min, q_max = axs[0, 1].get_ylim()
        x_min, x_max = axs[1, 0].get_ylim()
        y_min, y_max = axs[1, 1].get_ylim()
        r_min, r_max = axs[2, 0].get_ylim()
        s_min, s_max = axs[2, 1].get_ylim()
        axs[0, 0].set_ylim(0, 2 * p0)
        axs[0, 1].set_ylim(0, 2 * q0)
        axs[1, 0].set_ylim(min(x_min, y_min), max(x_max, y_max))
        axs[1, 1].set_ylim(min(x_min, y_min), max(x_max, y_max))
        axs[2, 0].set_ylim(r_min, r_max)
        axs[2, 1].set_ylim(s_min, s_max)
    except Exception as exception:
        print("graph fail")
        raise exception
    print("graph success")
    handles, labels = axs[2, 0].get_legend_handles_labels()
    fig.legend(handles, labels, mode='expand', loc='outside lower center')
    fig.suptitle("A graph of the unreduced base model with constant P and Q.")
    plt.show()

Select model

In [13]:
def f(t, u, coeffs):
    p, q, x, y, r, s = u
    k1, k2, k3, k4 = coeffs
    return np.array([
        0,
        0,
        k1 * p - k2 * q * x + k3 * x * x * y - k4 * x,
        k2 * q * x - k3 * x * x * y,
        k2 * q * x,
        k4 * x
    ])

Specify parameters

In [14]:
u0 = [0.59, 0.56, 0.83, 1.25, 0, 0]  # list of starting values of the variables; first part of the parameters.
coeffs = [0.19, 1.07, 0.85, 0.22]  # list of coefficients for reaction speeds; second part of the parameters.
time_bound = 200  # cutoff point in time to stop the simulation at, or None for the default value of 50.
granularity = None  # number of points in time to actually log the values at (not counting t=0),
# or None to let the solver itself decide for us.

Execute simulation

In [23]:
result = execute_simulation(f=f, u0=u0, coeffs=coeffs, time_bound=time_bound, granularity=granularity)
success_out, u0_out, coeffs_out, time_bound_out, granularity_out, sol_out = result

modified_u0=[1, 1, 1, 1, 0, 0]
modified_coeffs=[0.2, 0.3]
success


Execute visualization

In [ ]:
if success_out:
    visualize_simulation(u0_out, coeffs_out, time_bound_out, granularity_out, sol_out)
else:
    print("simulation failed. Stopping and printing failed settings.")
    print(u0_out, "\n", coeffs_out)